In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tf
import scipy.signal as sg

import j_py_sad_correlation as jps

# Python libraries
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import curve_fit
import tifffile as tf

# Custom libraries
import optical_gating_analysis as OG

In [2]:
def v_fitting(y_1, y_2, y_3):
    # Fit using a symmetric 'V' function, to find the interpolated minimum for three datapoints y_1, y_2, y_3,
    # which are considered to be at coordinates x=-1, x=0 and x=+1
    if y_1 < y_2 or y_3 < y_2:
        return 0, 0

    if y_1 > y_3:
        x = 0.5 * (y_1 - y_3) / (y_1 - y_2)
        y = y_2 - x * (y_1 - y_2)
    else:
        x = 0.5 * (y_1 - y_3) / (y_3 - y_2)
        y = y_2 + x * (y_3 - y_2)


    return x, y

In [3]:
oog = OG.BasicOpticalGating()
oog.sequence_manager.set_source(r"D:\Data\2012-06-20 13.34.11 vid 2x2 multi phase single plane\brightfield\*tif")
oog.run()

Setting source to D:\Data\2012-06-20 13.34.11 vid 2x2 multi phase single plane\brightfield\*tif
Getting reference sequence
Getting SADs
Getting phases


In [4]:
sequence = tf.imread(oog.sequence_manager.sequence_src)
diffs = np.zeros((sequence.shape[0], sequence.shape[0]))

for i in range(sequence.shape[0]):
    frame = sequence[i]
    pastFrames = sequence
    diffs[i, :] = jps.sad_with_references(frame, pastFrames)

In [5]:
peak_loc = []
phase = []

for i in range(sequence.shape[0] - 1):
    peaks = sg.find_peaks(-diffs[i], prominence = 0.5e5, width = 0)[0]



    for peak in peaks:
        sad_left = diffs[i][peak - 1]
        sad_center = diffs[i][peak]
        sad_right = diffs[i][peak + 1]
        subframe = v_fitting(sad_left, sad_center, sad_right)[0]

        loc = peak + subframe
        
        peak_loc.append(loc)
        phase.append(oog.phases[i])

    #peak_locs.append(peak_loc)
    #phases.append(phase)



In [6]:
delta_phases = []
for i in range(1, phase.shape[0]):
    delta_phase = (phase[i] - phase[i - 1])# / (peak_loc[i] - peak_loc[i - 1])
    if delta_phase < -oog.sequence_manager.reference_period / 2:
        delta_phase += oog.sequence_manager.reference_period

    delta_phases.append(delta_phase)

idx = np.argsort(peak_loc[1::])

plt.scatter(range(400), np.array(delta_phases)[idx][0:400])

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
plt.figure(figsize = (100,100))
plt.scatter(peak_loc, phase, s = 5)
plt.scatter(range(len(oog.phases)), oog.phases)
plt.show()